# 🤖 Models

This notebook shows how to use the various network architectures defined in this project.

## Setup

---

Let's install some necessary dependencies and set global variables.

In [ ]:
%reload_ext autoreload
%autoreload 2

In [ ]:
# Autoroot
import autorootcwd

In [ ]:
# Imports
import torch
from hydra import compose, initialize
from hydra.utils import instantiate

from src.models.net import FFNet, ConvNet, UNet, AutoTranslateNet

## Models


### FFNet

The `FFNet` is a simple linear encoder-decoder network.

In [ ]:
# Initialise `FFNet`
net = FFNet(input_output_size=32*32, hidden_dims=[256, 128])
net

In [ ]:
# Forward pass
x = torch.randn(1, 1, 32, 32)
y = net(x)

assert x.shape == y.shape
x.shape, y.shape

### ConvNet


A simple convolutional encoder-decoder neural network (without skip connections)

In [ ]:
# Initialise `ConvNet`
net = ConvNet(input_output_channels=3, hidden_channels=[64, 128, 256])
net

In [ ]:
# Forward pass
x = torch.randn(1, 3, 32, 32)
y = net(x)

assert x.shape == y.shape
x.shape, y.shape

### UNet

In [ ]:
# Initalise UNet
net = UNet(input_output_channels=3, hidden_channels=[64, 128, 256, 512])
net

In [ ]:
# Forward pass
x = torch.randn(1, 3, 32, 32)
y = net(x)

assert x.shape == y.shape
x.shape, y.shape

### AutoTranslateNet

The AutoTranslateNet is an implementation of the network seen in
'Semi-Supervised Raw-to-Raw mapping' https://arxiv.org/pdf/2106.13883

The network consists of two auto-encoders, one for the digital domain and
one for the film domain.

There is then a translation network that is trained to map the latent space
of the source domain auto-encoder to the latent space of the target domain
auto-encoder.

In [ ]:
# Initialise `AutoTranslateNet`
net = AutoTranslateNet(input_output_channels=3, hidden_channels=[64, 128, 256, 512])
net

In [ ]:
# Forward pass
film = torch.randn(1, 3, 32, 32)
digital = torch.randn(1, 3, 32, 32)
paired = (torch.randn(1, 3, 32, 32), torch.randn(1, 3, 32, 32))

(
    digital_reconstructed,
    film_reconstructed,
    digital_to_film,
    film_to_digital,
    paired_encoder_representations,
) = net(digital, film, paired)

## Hydra

In [ ]:
for model in ["ffnn", "conv", "unet", "auto-translate"]:
        with initialize(version_base=None, config_path="../configs/model/net", job_name="nets"):
                cfg = compose(config_name=model)
                net = instantiate(cfg)
                print(f"✅ Loaded {cfg._target_}")